In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import geocoder
from sklearn.linear_model import LogisticRegression

In [2]:
trees = pd.read_csv('../data/tree_dogs.csv')

In [ ]:
def get_latlong(address):
    g = geocoder.arcgis(address)
    return g.latlng

In [24]:
dogs_by_zip = trees.groupby('zipcode', as_index=False).mean()[['zipcode', 'dogs_near_tree']]
dogs_by_zip['dogs_near_tree'].fillna(0, inplace=True)
dogs_by_zip.head()

,zipcode,dogs_near_tree
0,83,0.0
1,10001,870.0
2,10002,1105.0
3,10003,1515.0
4,10004,132.0


In [25]:
health_map = {'Dead':0, 'Poor': 1, 'Fair': 2, 'Good': 3}
weight = trees.groupby(['zipcode', 'health'], as_index=False).count()
weight['health_grade'] = weight['health'].map(health_map)
weight['weighted_health'] = weight['health_grade'] * weight['address']
weight = weight.groupby('zipcode', as_index=False).sum()[['zipcode', 'weighted_health']]
df = weight.merge(dogs_by_zip, on='zipcode')

,zipcode,weighted_health,dogs_near_tree
0,83,2741,0.0
1,10001,2388,870.0
2,10002,5857,1105.0
3,10003,5264,1515.0
4,10004,328,132.0


In [40]:
mapit = folium.Map( location=[40.7128, -74.0060], tiles="OpenStreetMap", zoom_start=12)
# Loop through the results
for i in range(df.shape[0]):
    lat = geocoder.arcgis(df.iloc[i]['zipcode']).latlng[0]
    lng = geocoder.arcgis(df.iloc[i]['zipcode']).latlng[1]
    folium.Circle(location = [lat, lng], 
                  radius= df.iloc[i]['weighted_health']*0.1,
                  color = 'green',
                  fill = True,
                  fill_color = 'green').add_to(mapit)
    folium.Circle(location = [lat, lng], 
                  radius= df.iloc[i]['dogs_near_tree']*0.1,
                  color = 'crimson', 
                  fill=True,
                  fill_color='crimson').add_to(mapit)
        
mapit

In [42]:
mapit.save('../data/mapit.html')

In [43]:
![image](../images/map_dogs_healthytrees.png)

/bin/sh: -c: line 0: syntax error near unexpected token `../images/map_dogs_healthytrees.png'
/bin/sh: -c: line 0: `[image](../images/map_dogs_healthytrees.png)'
